# Carregar e Explorar os Dados

In [ ]:
import pandas as pd
# Carregar os dados
data = pd.read_csv('agendamentos.csv')

# Exibir as primeiras linhas dos dados
print(data.head())

# Informações gerais sobre os dados
print(data.info())

# Remover espaços em branco extras
data['DIA_DA_SEMANA'] = data['DIA_DA_SEMANA'].astype(str).str.strip()
data['TIPO_CONVENIO'] = data['TIPO_CONVENIO'].astype(str).str.strip()
data['TIPO_PRESENCA'] = data['TIPO_PRESENCA'].astype(str).str.strip()

# Verificar valores únicos nas colunas relevantes
print(data['TIPO_CONVENIO'].unique())
print(data['TIPO_PRESENCA'].unique())
print(data['DIA_DA_SEMANA'].unique())


                                 DS_ITEM_AGENDAMENTO  IDADE TIPO_CONVENIO  \
0                 AUDIOMETRIA TONAL DE DISCRIMINACAO     60      CONVENIO   
1  PESQUISA DE POTENCIAIS AUDITIVOS DE TRONCO CER...     66      CONVENIO   
2  PESQUISA DE POTENCIAIS AUDITIVOS DE TRONCO CER...     10      CONVENIO   
3                                 ELETROCOCLEOGRAFIA     64      CONVENIO   
4                                 ELETROCOCLEOGRAFIA     25      CONVENIO   

   NR_DDD_CELULAR TIPO_PRESENCA DATA_AGENDAMENTO DIA_DA_SEMANA  HORA_AGENDADO  
0              11      PRESENTE       06/01/2026     Tuesday               17  
1              11      PRESENTE       20/01/2023     Friday                 8  
2              11      PRESENTE       20/01/2023     Friday                 9  
3              11      PRESENTE       20/01/2023     Friday                10  
4              11      PRESENTE       27/01/2023     Friday                 8  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89402 e

# Pré-processamento dos dados


In [ ]:
# Conversão de variáveis categóricas para numéricas
data['TIPO_CONVENIO'] = data['TIPO_CONVENIO'].map({'CONVENIO': 0, 'PARTICULAR': 1})
data['TIPO_PRESENCA'] = data['TIPO_PRESENCA'].map({'PRESENTE': 0, 'FALTA': 1})
data['DIA_DA_SEMANA'] = data['DIA_DA_SEMANA'].map({
    'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
    'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6
})

# Verificar se há valores NaN após o mapeamento
print(data[['TIPO_CONVENIO', 'TIPO_PRESENCA', 'DIA_DA_SEMANA']].isna().sum())

# Remover linhas com valores NaN
data = data.dropna(subset=['TIPO_CONVENIO', 'TIPO_PRESENCA', 'DIA_DA_SEMANA'])

# Conversão de colunas de data e hora
data['DATA_AGENDAMENTO'] = pd.to_datetime(data['DATA_AGENDAMENTO'], format='%d/%m/%Y')
data['HORA_AGENDADO'] = data['HORA_AGENDADO'].astype(int)

# Criação de uma nova feature para identificar períodos do dia
def categorize_time(hour):
    if hour < 12:
        return 0  # Manhã
    elif 12 <= hour < 18:
        return 1  # Tarde
    else:
        return 2  # Noite

data['PERIODO'] = data['HORA_AGENDADO'].apply(categorize_time)

# Feature para identificar se o DDD é 11
data['DDD_11'] = data['NR_DDD_CELULAR'].apply(lambda x: 1 if x == 11 else 0)

# Seleção de features e target
features = data[['TIPO_CONVENIO', 'DIA_DA_SEMANA', 'PERIODO', 'DDD_11', 'IDADE']]
target = data['TIPO_PRESENCA']

print(features.head())
print(target.head())

# Verificar tipos de dados
print(features.dtypes)
print(target.dtypes)

# Garantir que o target seja de tipo inteiro
target = target.astype(int)


TIPO_CONVENIO    0
TIPO_PRESENCA    0
DIA_DA_SEMANA    0
dtype: int64
   TIPO_CONVENIO  DIA_DA_SEMANA  PERIODO  DDD_11  IDADE
0              0              1        1       1     60
1              0              4        0       1     66
2              0              4        0       1     10
3              0              4        0       1     64
4              0              4        0       1     25
0    0
1    0
2    0
3    0
4    0
Name: TIPO_PRESENCA, dtype: int64
TIPO_CONVENIO    int64
DIA_DA_SEMANA    int64
PERIODO          int64
DDD_11           int64
IDADE            int64
dtype: object
int64


# Implementação e Execução do Algoritmo Genético

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Função de fitness
def fitness(solution):
    n_estimators, max_depth, min_samples_split = solution
    model = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=max(int(max_depth), 1),  # Garantir que max_depth seja pelo menos 1
        min_samples_split=max(int(min_samples_split), 2),  # Garantir que min_samples_split seja pelo menos 2
        random_state=42
    )
    try:
        model.fit(X_train, y_train)
    except ValueError as e:
        print(f'Erro durante o ajuste do modelo: {e}')
        return 0
    predictions = model.predict(X_test)
    return accuracy_score(y_test, predictions)

# Criação da população inicial
def create_population(size):
    population = np.random.rand(size, 3)
    population[:, 0] = population[:, 0] * 100  # n_estimators no intervalo [1, 100]
    population[:, 1] = population[:, 1] * 19 + 1  # max_depth no intervalo [1, 20]
    population[:, 2] = population[:, 2] * 9 + 2  # min_samples_split no intervalo [2, 11]
    return population

# Seleção
def selection(population, fitnesses, num_parents):
    parents_idx = np.argsort(fitnesses)[-num_parents:]
    return population[parents_idx]

# Crossover
def crossover(parents, offspring_size):
    offspring = np.empty((offspring_size, parents.shape[1]))
    for k in range(offspring_size):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        crossover_point = np.random.randint(0, parents.shape[1])
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

# Mutação
def mutation(offspring):
    for idx in range(offspring.shape[0]):
        random_value = np.random.uniform(-1.0, 1.0, 1)[0]  # Garantir que random_value seja um valor escalar
        mutation_idx = np.random.randint(0, offspring.shape[1])
        offspring[idx, mutation_idx] += random_value
        # Garantir que os valores mutados estão dentro dos intervalos permitidos
        if mutation_idx == 0:
            offspring[idx, mutation_idx] = np.clip(offspring[idx, mutation_idx], 1, 100)
        elif mutation_idx == 1:
            offspring[idx, mutation_idx] = np.clip(offspring[idx, mutation_idx], 1, 20)
        else:
            offspring[idx, mutation_idx] = np.clip(offspring[idx, mutation_idx], 2, 11)
    return offspring

# Parâmetros do AG
population_size = 20
num_generations = 50
num_parents = 10
n_jobs = -1  # Usar todos os processadores disponíveis

# Inicialização da população
population = create_population(population_size)

# Evolução
for generation in range(num_generations):
    fitnesses = Parallel(n_jobs=n_jobs)(delayed(fitness)(ind) for ind in population)
    parents = selection(population, fitnesses, num_parents)
    offspring_size = population_size - num_parents
    offspring = crossover(parents, offspring_size)
    offspring = mutation(offspring)
    population[:num_parents, :] = parents
    population[num_parents:, :] = offspring

    best_solution_idx = np.argmax(fitnesses)
    print(f'Geração {generation}: Melhor Fitness = {fitnesses[best_solution_idx]}')

# Melhor solução final
best_solution_idx = np.argmax(fitnesses)
best_solution = population[best_solution_idx]
print(f'Melhor solução: {best_solution}')


Geração 0: Melhor Fitness = 0.9025017710003356
Geração 1: Melhor Fitness = 0.9025017710003356
Geração 2: Melhor Fitness = 0.9025017710003356
Geração 3: Melhor Fitness = 0.9025390552179262
Geração 4: Melhor Fitness = 0.9025390552179262
Geração 5: Melhor Fitness = 0.9025390552179262
Geração 6: Melhor Fitness = 0.9025390552179262
Geração 7: Melhor Fitness = 0.9025390552179262
Geração 8: Melhor Fitness = 0.9025390552179262
Geração 9: Melhor Fitness = 0.9025390552179262
Geração 10: Melhor Fitness = 0.9025390552179262
Geração 11: Melhor Fitness = 0.9025390552179262
Geração 12: Melhor Fitness = 0.9025390552179262
Geração 13: Melhor Fitness = 0.9025390552179262
Geração 14: Melhor Fitness = 0.9025390552179262
Geração 15: Melhor Fitness = 0.9025390552179262
Geração 16: Melhor Fitness = 0.9025390552179262
Geração 17: Melhor Fitness = 0.9025390552179262
Geração 18: Melhor Fitness = 0.9025390552179262
Geração 19: Melhor Fitness = 0.9025390552179262
Geração 20: Melhor Fitness = 0.9025390552179262
Ge

# Avaliação do Modelo


In [ ]:
best_n_estimators, best_max_depth, best_min_samples_split = best_solution

# Treinamento do modelo com os melhores parâmetros
best_model = RandomForestClassifier(
    n_estimators=int(best_n_estimators),
    max_depth=int(best_max_depth),
    min_samples_split=int(best_min_samples_split),
    random_state=42
)
best_model.fit(X_train, y_train)
best_predictions = best_model.predict(X_test)

# Avaliação
accuracy = accuracy_score(y_test, best_predictions)
print(f'Accuracy: {accuracy}')


Accuracy: 0.9025390552179262
